In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:

from dataclasses import dataclass
import asyncio 
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 

# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)

In [18]:
from pydantic import BaseModel
from typing import Literal,Union
from agents import Agent 

class CategoryOnly(BaseModel):
    category: Literal[
        "OrderTracking", 
        "CustomerComplaint",
        "ComplaintStatus",
        "KnowledgeBaseQA", 
        "ProductRecommendation",
    ]
class DirectReply(BaseModel):
    message: str

CategorizerType = Union[CategoryOnly, DirectReply]

categorizer = Agent(
    name="categorizer",
    instructions = (
    "You are an intelligent orchestration agent responsible for routing user queries "
    "to the most appropriate specialized sub-agent based on the user's intent.\n\n"

    "Your job is to either:\n"
    "- Return a category in JSON like { \"category\": \"SubAgentName\" }, OR\n"
    "- Reply directly using { \"message\": \"<response>\" } if the query is off-topic or just a greeting.\n\n"

    "### Categories:\n"
    "1. **OrderTracking** – For all queries related to a user's orders, including:\n"
    "   - Checking the status or shipping progress using a tracking ID.\n"
    "   - Viewing a user's full order history using their email address.\n\n"

    "2. **CustomerComplaint** – When the user wants to raise a concern, report an issue, or file a complaint about a shoe or delivery. Typically requires a `tracking_id`.\n\n"

    "3. **ComplaintStatus** – When the user wants to check the status or resolution progress of a previously submitted complaint. Requires a `complaint_id`.\n\n"

    "4. **KnowledgeBaseQA** – For general questions about store policies, payments, returns, warranties, product care, or shoe sizing. This is powered by a Retrieval-Augmented Generation (RAG) system.\n\n"

    "5. **ProductRecommendation** – When the user requests personalized shoe suggestions based on filters like size, color, price, product type, or discounts.\n\n"

    "### Classification Examples:\n"
    '- "Can you track my shipment?" → { "category": "OrderTracking" }\n'
    '- "What shoes did I buy last month?" → { "category": "OrderTracking" }\n'
    '- "Where is my order TRK00123?" → { "category": "OrderTracking" }\n'
    '- "I received the wrong size" → { "category": "CustomerComplaint" }\n'
    '- "What’s the update on my complaint from last week?" → { "category": "ComplaintStatus" }\n'
    '- "How do I return a damaged product?" → { "category": "KnowledgeBaseQA" }\n'
    '- "Suggest white sneakers in size 10 under $100" → { "category": "ProductRecommendation" }\n\n'

    "### Special Cases:\n"
    "- If the user's question is outside your domain (e.g., news, politics, programming, ChatGPT), respond with:\n"
    '  { "message": "I’m ArmanStore’s assistant. I can help with questions about our shoes, orders, complaints, or searching for a product using an uploaded image." }\n\n'

    "- If the user greets you or starts with a friendly message (e.g., 'Hi', 'Hello', 'Hey', 'How are you?', 'How’s it going?'), respond with:\n"
    '  { "message": "Hey there! 😊 I\'m your ArmanStore assistant — ready to help you with anything related to your orders, complaints, or even finding the perfect shoes. How can I assist you today?" }\n\n'

    "**Never guess categories. Only return what's accurate. Do not hallucinate or make assumptions.**"
), 
    model=model,
    output_type=CategorizerType,
)

In [22]:
from agents import function_tool,Agent 

@function_tool(strict_mode=False)
def recommend_product(
    size: float,
    gender:str=None,
    color: str = None,
    discount: float = None,
    price: float = None,
    for_use: str = None, 
):
    """
    Recommends a product based on customer preferences.

    Args:
        size (float): Customer's preferred shoe size in US or EU sizing (e.g., 9.5 or 42.5).
        gender (str, optional): One of ["MALE", "FEMALE", "BOTH"]
        color (str, optional): Preferred color of the shoe (e.g., "Black", "White").
        discount (float, optional): Minimum discount required, as a percentage (e.g., 10.0 for 10%).
        price (float, optional): Maximum price the user is willing to pay (e.g., 100.0).
        for_use (str, optional): Intended use of the product, e.g., "Running", "Basketball", etc. 

    Returns:
        str: A message recommending the most suitable product or a message indicating no matches found.
    """
  
    return 'resp'
 

ProductRecommendation = Agent(
    name="user_recommendation_agent",
      instructions=(
        "You are a smart shoe recommendation assistant at **ArmanStore**.\n\n"
        "Your task is to help customers find the most suitable shoe products based on their preferences.\n\n"
        
        "**Required fields:**\n"
        "- `size` (US or EU shoe size)\n\n"
        
        "**Optional filters:**\n"
        "- `gender`: Must be one of `MALE`, `FEMALE`, `BOTH` (case-insensitive)\n"
        "- `color` (e.g., black,white,university red)\n"
        "- `discount` (minimum discount the user prefers)\n\n"
        "- `price` (maximum price the user is willing to pay)\n"
        "- `for_use` (e.g., Training, Running, Casual)\n"   
        
        "**For Use Matching Rule:**\n"
        "If the user provides a `for_use` preference, match it exactly against the following values:\n"
        "`Basketball`, `Casual`, `Casual Play`, `Casual Wear`, `Casual, Everyday`, `Casual, Running`, `Competition`, `Court Sports`, `Cross Training`, `Fitness`, `Hiking`, `Indoor Soccer`, `Indoor Sports`, `Indoor and Outdoor`, `Lifestyle`, `Marathon Running`, `Multi-sport`, `Play/Indoor Sports`, `Playground, Running`,`Road Running`, `Running`, `Soccer`, `Sports Training`, `Tennis`, `Trail Running`, `Training`, `Walking`,`Walking, Everyday`, `Water Activities`, `Weight Training`, `Weightlifting`, `Wrestling`\n"
        
        "**Size Interpretation Rule:**\n"
        "- The system supports US sizes from 1 to 15 and EU sizes from 33 to 47.\n"
        "- If the user says things like 'US 10', 'size 10', or just a number between 1–15, treat it as `size`.\n"
        "- If the user says 'EU 44', 'size 42', or mentions a number between 32–49, treat it as `size`.\n"
        "- If the size is outside supported ranges, ignore it or ask the user to provide a valid size.\n\n"
        
        "**Behavior Rules:**\n"
        "- If size is missing or invalid, politely ask the user to provide a valid shoe size.\n"
        "- If gender is provided but invalid, inform the user of allowed values.\n"
        "- Always call the `recommend_product` tool with all available valid inputs.\n"
        "- If no matching products found, inform the user.\n"
        "- If the message is vague, such as: 'I want to buy shoes like my friend/teacher/colleague', and you can't detect the size, reply:\n"
        "  'To help you find the right product, please provide your shoe size — or upload a clear side-view photo of your [friend’s/teacher’s/colleague’s] shoes.'\n"
 
    ),
    model=model,
    tools=[recommend_product], 
    tool_use_behavior='stop_on_first_tool'
)


In [24]:
from agents import SQLiteSession
session = SQLiteSession("armanstore_123")
result = await Runner.run(
    ProductRecommendation,
    "my shoes size is 12",
    session=session,
)
print(result.final_output)  # "San Francisco"

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332D50>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332D50>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332D50>
Running agent user_recommendation_agent (turn 1)
Running agent user_recommendation_agent (turn 1)
Running agent user_recommendation_agent (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013D3834D0>
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013D3834D0>
Tracing is disabled. Not creating span <agents

In [26]:
from agents import SQLiteSession
session = SQLiteSession("armanstore_123")
result = await Runner.run(
    ProductRecommendation,
    "i want to buy shoes",
    session=session,
)
print(result.final_output)  # "San Francisco"

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D3E7F70>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D3E7F70>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D3E7F70>
Running agent user_recommendation_agent (turn 1)
Running agent user_recommendation_agent (turn 1)
Running agent user_recommendation_agent (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013D3834D0>
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013D3834D0>
Tracing is disabled. Not creating span <agents

In [27]:
!pip install psutil



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import psutil

memory = psutil.virtual_memory()

print(f"Total     : {memory.total / (1024 ** 3):.2f} GB")
print(f"Available : {memory.available / (1024 ** 3):.2f} GB")
print(f"Used      : {memory.used / (1024 ** 3):.2f} GB")
print(f"Free      : {memory.free / (1024 ** 3):.2f} GB")
print(f"Percent   : {memory.percent} %")


Total     : 15.35 GB
Available : 3.54 GB
Used      : 11.81 GB
Free      : 3.54 GB
Percent   : 77.0 %


In [32]:
import psutil

# Get all processes and sort by memory usage
processes = sorted(psutil.process_iter(['pid', 'name', 'memory_info']),
                   key=lambda p: p.info['memory_info'].rss if p.info['memory_info'] else 0,
                   reverse=True)

print(f"{'PID':<10}{'Memory (MB)':<15}{'Process Name'}")
print("-" * 40)

# Show top 10 memory-consuming processes
for proc in processes[:100]:
    try:
        mem_mb = proc.info['memory_info'].rss / (1024 ** 2)
        print(f"{proc.info['pid']:<10}{mem_mb:<15.2f}{proc.info['name']}")
    except (psutil.NoSuchProcess, psutil.AccessDenied):
        continue


PID       Memory (MB)    Process Name
----------------------------------------
21480     1217.27        python.exe
3312      988.51         MemCompression
26220     416.00         Code.exe
14228     397.94         Code.exe
19040     369.92         Code.exe
17404     335.46         chrome.exe
20212     325.56         Code.exe
16968     321.58         Code.exe
4412      321.23         chrome.exe
27240     263.38         explorer.exe
23476     260.70         Code.exe
9948      259.22         chrome.exe
23508     218.55         MsMpEng.exe
11060     213.04         chrome.exe
23676     190.47         chrome.exe
15092     179.27         chrome.exe
27620     177.51         chrome.exe
22572     164.38         chrome.exe
12560     158.77         chrome.exe
22452     154.02         Taskmgr.exe
3428      142.29         Code.exe
18480     141.97         msedgewebview2.exe
12096     140.57         chrome.exe
18636     140.16         chrome.exe
9172      139.63         chrome.exe
11728     139.48   

In [11]:
from agents import Agent, Runner, SQLiteSession

# Create agent
agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
    model=model
)

# Create a session instance with a session ID
session = SQLiteSession("conversation_123")

# First turn
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session,
)
print(result.final_output)  # "San Francisco"


San Francisco.



In [9]:

# Second turn - agent automatically remembers previous context
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"


California.



In [13]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging()
# Also works with synchronous runner
result = Runner.run_sync(
    agent,
    "What's the population?",
    session=session
)
print(result.final_output)  # "Approximately 39 million"

Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D333250>
Running agent Assistant (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013D2839B0>
[
  {
    "content": "Reply very concisely.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "What city is the Golden Gate Bridge in?"
  },
  {
    "role": "assistant",
    "content": "San Francisco.\n"
  },
  {
    "role": "user",
    "content": "What's the population?"
  },
  {
    "role": "assistant",
    "content": "Around 800,000.\n"
  },
  {
    "role": "user",
    "content": "What's the population?"
  }
]
Tools:
[]
Stream: False
Tool choice: NOT_GIVEN
Response format: NOT_GIVEN

LLM resp:
{
  "content": "Around 800,000.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null

In [14]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging()
# Also works with synchronous runner
result = Runner.run_sync(
    agent,
    "hi",
    session=session
)
print(result.final_output)  # "Approximately 39 million"

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013CF1B660>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013CF1B660>
Running agent Assistant (turn 1)
Running agent Assistant (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013C882270>
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013C882270>
[
  {
    "content": "Reply very concisely.",
    "role": "system"
  },
  {
    "role": "user",
    "content": "What city is the Golden Gate Bridge in?"
  },
  {
    "role": "assistant",
    "content": "San Francisco.\n"
  },
  {
    "role": "user",
    "content": "What's the population?"
  },
  {
    "role": "assistan

In [15]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging()
# Also works with synchronous runner
result = Runner.run_sync(
    agent,
    "What's the population?",
    session=session
)
print(result.final_output)  # "Approximately 39 million"

Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Setting current trace: no-op
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332DF0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332DF0>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x000002013D332DF0>
Running agent Assistant (turn 1)
Running agent Assistant (turn 1)
Running agent Assistant (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013CAA3830>
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x000002013CAA3830>
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 